# T81-558: Applications of Deep Neural Networks
**Module 13: Advanced/Other Topics**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 13 Video Material

Main video lecture:

* **Part 13.1: Deploying a Model to AWS** [[Video]]() [[Notebook]](t81_558_class_13_01_flask.ipynb)
* Part 13.2: Flask and Deep Learning Web Services [[Video]]() [[Notebook]](t81_558_class_13_02_cloud.ipynb)
* Part 13.3: AI at the Edge: Using Keras on a Mobile Device [[Video]]() [[Notebook]](t81_558_class_13_03_web.ipynb)
* Part 13.4: When to Retrain Your Neural Network [[Video]]() [[Notebook]](t81_558_class_13_04_edge.ipynb)
* Part 13.5: Using a Keras Deep Neural Network with a Web Application [[Video]]() [[Notebook]](t81_558_class_13_05_retrain.ipynb)


# Part 13.1: Deploying a Model to AWS

* [Flask Quickstart](https://flask.palletsprojects.com/en/1.0.x/quickstart/)

### Flask Hello World

In [5]:
from werkzeug.wrappers import Request, Response
from flask import Flask

app = Flask(__name__)

@app.route("/")
def hello():
    return "Hello World!"

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)

 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Jul/2019 11:53:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2019 11:53:13] "GET /favicon.ico HTTP/1.1" 404 -


### MPG Flask

Sample JSON

```
{
  "cylinders": 8, 
  "displacement": 300,
  "horsepower": 78, 
  "weight": 3500,
  "acceleration": 20, 
  "year": 76,
  "origin": 1
}
```

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", 
    na_values=['NA', '?'])

cars = df['name']

# Handle missing value
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].median())

# Pandas to Numpy
x = df[['cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'year', 'origin']].values
y = df['mpg'].values # regression

# Split into validation and training sets
x_train, x_test, y_train, y_test = train_test_split(    
    x, y, test_size=0.25, random_state=42)

# Build the neural network
model = Sequential()
model.add(Dense(25, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(10, activation='relu')) # Hidden 2
model.add(Dense(1)) # Output
model.compile(loss='mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto',
        restore_best_weights=True)
model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)

Train on 298 samples, validate on 100 samples
Epoch 1/1000
298/298 - 0s - loss: 159630.0470 - val_loss: 116415.4753
Epoch 2/1000
298/298 - 0s - loss: 95357.2841 - val_loss: 64835.0947
Epoch 3/1000
298/298 - 0s - loss: 49977.8136 - val_loss: 30844.7720
Epoch 4/1000
298/298 - 0s - loss: 22480.1594 - val_loss: 11900.8987
Epoch 5/1000
298/298 - 0s - loss: 7921.1765 - val_loss: 3411.9030
Epoch 6/1000
298/298 - 0s - loss: 2041.2930 - val_loss: 595.4836
Epoch 7/1000
298/298 - 0s - loss: 320.6314 - val_loss: 99.5764
Epoch 8/1000
298/298 - 0s - loss: 115.4243 - val_loss: 142.3426
Epoch 9/1000
298/298 - 0s - loss: 153.6348 - val_loss: 154.5919
Epoch 10/1000
298/298 - 0s - loss: 142.9074 - val_loss: 123.6056
Epoch 11/1000
298/298 - 0s - loss: 117.0491 - val_loss: 101.9533
Epoch 12/1000
298/298 - 0s - loss: 106.5719 - val_loss: 95.7690
Epoch 13/1000
298/298 - 0s - loss: 105.1322 - val_loss: 95.1105
Epoch 14/1000
298/298 - 0s - loss: 105.5002 - val_loss: 94.8752
Epoch 15/1000
298/298 - 0s - loss: 1

Epoch 129/1000
298/298 - 0s - loss: 70.9175 - val_loss: 60.8782
Epoch 130/1000
298/298 - 0s - loss: 70.9808 - val_loss: 60.4282
Epoch 131/1000
298/298 - 0s - loss: 70.5090 - val_loss: 58.9432
Epoch 132/1000
298/298 - 0s - loss: 70.2090 - val_loss: 58.8502
Epoch 133/1000
298/298 - 0s - loss: 69.8123 - val_loss: 60.0325
Epoch 134/1000
298/298 - 0s - loss: 69.7477 - val_loss: 59.0148
Epoch 135/1000
298/298 - 0s - loss: 69.5066 - val_loss: 57.5705
Epoch 136/1000
298/298 - 0s - loss: 69.0220 - val_loss: 58.1045
Epoch 137/1000
298/298 - 0s - loss: 68.7312 - val_loss: 57.6961
Epoch 138/1000
298/298 - 0s - loss: 68.5292 - val_loss: 57.5752
Epoch 139/1000
298/298 - 0s - loss: 68.1027 - val_loss: 57.7373
Epoch 140/1000
298/298 - 0s - loss: 67.8267 - val_loss: 56.8663
Epoch 141/1000
298/298 - 0s - loss: 67.8240 - val_loss: 56.0898
Epoch 142/1000
298/298 - 0s - loss: 67.4671 - val_loss: 56.7801
Epoch 143/1000
298/298 - 0s - loss: 67.2163 - val_loss: 56.9900
Epoch 144/1000
298/298 - 0s - loss: 66.8

298/298 - 0s - loss: 44.2277 - val_loss: 35.6553
Epoch 258/1000
298/298 - 0s - loss: 43.7172 - val_loss: 37.5211
Epoch 259/1000
298/298 - 0s - loss: 43.6284 - val_loss: 35.1197
Epoch 260/1000
298/298 - 0s - loss: 43.8555 - val_loss: 36.6300
Epoch 261/1000
298/298 - 0s - loss: 43.2431 - val_loss: 35.1653
Epoch 262/1000
298/298 - 0s - loss: 43.0753 - val_loss: 35.8108
Epoch 263/1000
298/298 - 0s - loss: 43.6783 - val_loss: 34.9497
Epoch 264/1000
298/298 - 0s - loss: 42.7993 - val_loss: 34.9008
Epoch 265/1000
298/298 - 0s - loss: 43.0500 - val_loss: 35.7064
Epoch 266/1000
298/298 - 0s - loss: 42.8319 - val_loss: 36.6704
Epoch 267/1000
298/298 - 0s - loss: 42.8799 - val_loss: 34.2744
Epoch 268/1000
298/298 - 0s - loss: 42.0481 - val_loss: 35.9306
Epoch 269/1000
298/298 - 0s - loss: 41.7531 - val_loss: 33.8539
Epoch 270/1000
298/298 - 0s - loss: 42.0292 - val_loss: 34.4626
Epoch 271/1000
298/298 - 0s - loss: 41.7447 - val_loss: 34.4374
Epoch 272/1000
298/298 - 0s - loss: 41.5168 - val_loss:

Epoch 386/1000
298/298 - 0s - loss: 24.6707 - val_loss: 19.8317
Epoch 387/1000
298/298 - 0s - loss: 24.2970 - val_loss: 19.8942
Epoch 388/1000
298/298 - 0s - loss: 24.1268 - val_loss: 19.8651
Epoch 389/1000
298/298 - 0s - loss: 24.2748 - val_loss: 19.3862
Epoch 390/1000
298/298 - 0s - loss: 24.1217 - val_loss: 20.3736
Epoch 391/1000
298/298 - 0s - loss: 23.8309 - val_loss: 19.2344
Epoch 392/1000
298/298 - 0s - loss: 23.5380 - val_loss: 19.4152
Epoch 393/1000
298/298 - 0s - loss: 23.6220 - val_loss: 19.9142
Epoch 394/1000
298/298 - 0s - loss: 23.3728 - val_loss: 19.0153
Epoch 395/1000
298/298 - 0s - loss: 23.4537 - val_loss: 18.7974
Epoch 396/1000
298/298 - 0s - loss: 23.2030 - val_loss: 18.9056
Epoch 397/1000
298/298 - 0s - loss: 23.0214 - val_loss: 18.9373
Epoch 398/1000
298/298 - 0s - loss: 23.7718 - val_loss: 18.6109
Epoch 399/1000
298/298 - 0s - loss: 22.7860 - val_loss: 19.4805
Epoch 400/1000
298/298 - 0s - loss: 22.8862 - val_loss: 18.4385
Epoch 401/1000
298/298 - 0s - loss: 22.6

In [13]:
pred = model.predict(x_test)
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print(f"After load score (RMSE): {score}")

After load score (RMSE): 3.5492242937660206


In [14]:
model.save(os.path.join("./dnn/","mpg_model.h5"))

In [53]:
cols = [x for x in df.columns if x not in ('mpg','name')]

print("{")
for i,name in enumerate(cols):
    print(f'"{name}":{{"min":{df[name].min()},"max":{df[name].max()}}}{"," if i<(len(cols)-1) else ""}')
print("}")

{
"cylinders":{"min":3,"max":8},
"displacement":{"min":68.0,"max":455.0},
"horsepower":{"min":46.0,"max":230.0},
"weight":{"min":1613,"max":5140},
"acceleration":{"min":8.0,"max":24.8},
"year":{"min":70,"max":82},
"origin":{"min":1,"max":3}
}


In [12]:
import os
from tensorflow.keras.models import load_model
import numpy as np

model = load_model(os.path.join("./dnn/","mpg_model.h5"))
x = np.zeros( (1,7) )

x[0,0] = 8 # 'cylinders', 
x[0,1] = 400 # 'displacement', 
x[0,2] = 80 # 'horsepower', 
x[0,3] = 2000 # 'weight',
x[0,4] = 19 # 'acceleration', 
x[0,5] = 72 # 'year', 
x[0,6] = 1 # 'origin'


pred = model.predict(x)
float(pred[0])

19.488521575927734

### Flask MPG Client

In [3]:
import requests

json = {
  "cylinders": 8, 
  "displacement": 300,
  "horsepower": 78, 
  "weight": 3500,
  "acceleration": 20, 
  "year": 76,
  "origin": 1
}

r = requests.post("http://localhost:5000/api/mpg",json=json)
if r.status_code == 200:
    print("Success: {}".format(r.text))
else: print("Failure: {}".format(r.text))

Success: {
  "errors": [], 
  "id": "4eded983-67bb-4b84-b545-0cdcdbfdf99b", 
  "mpg": 18.122976303100586
}



In [7]:
import requests
response = requests.post('http://localhost:5000/api/image', files=dict(image='./photos/hickory.jpeg'))
if response.status_code == 200:
    print("Success: {}".format(response.text))
else: print("Failure: {}".format(response.text))

Failure: <!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN"
  "http://www.w3.org/TR/html4/loose.dtd">
<html>
  <head>
    <title>TypeError: The view function did not return a valid response. The function either returned None or ended without a return statement. // Werkzeug Debugger</title>
    <link rel="stylesheet" href="?__debugger__=yes&amp;cmd=resource&amp;f=style.css"
        type="text/css">
    <!-- We need to make sure this has a favicon so that the debugger does
         not by accident trigger a request to /favicon.ico which might
         change the application state. -->
    <link rel="shortcut icon"
        href="?__debugger__=yes&amp;cmd=resource&amp;f=console.png">
    <script src="?__debugger__=yes&amp;cmd=resource&amp;f=jquery.js"></script>
    <script src="?__debugger__=yes&amp;cmd=resource&amp;f=debugger.js"></script>
    <script type="text/javascript">
      var TRACEBACK = 48222288136,
          CONSOLE_MODE = false,
          EVALEX = true,
          E

In [3]:
!ls ./photos/hickory.jpeg

./photos/hickory.jpeg
